In [248]:
from pyomo.environ import *

In [249]:
solver = SolverFactory('gurobi')

In [250]:
r = ['solar', 'wind', 'power', 'water', 'hydrogen', 'oxygen']
o = ['PV', 'WF', 'PEM']

solar = 1.2  # - for power
wind = 1.3  # - for power
# for hydrogen
power = 3.9  # -
water = 2.95  # -
hydrogen = 1  # +
oxygen = 4.7  # +

In [251]:
m = ConcreteModel()
m.c = Var(['solar', 'wind', 'water'], within=NonNegativeReals)
m.d = Var(['hydrogen', 'oxygen'], within=NonNegativeReals)
m.p = Var(['power', 'hydrogen', 'oxygen'], within=NonNegativeReals)

m.con1 = Constraint(expr=m.p['hydrogen'] - m.d['hydrogen'] == 0)
m.con2 = Constraint(expr=m.p['oxygen'] - m.d['oxygen'] == 0)

m.con3 = Constraint(expr=m.c['water'] - water * m.p['hydrogen'] == 0)
m.con4 = Constraint(expr=m.d['oxygen'] - oxygen * m.p['hydrogen'] == 0)
m.con5 = Constraint(expr=m.p['power'] - power * m.p['hydrogen'] == 0)

m.con6 = Constraint(
    expr=(1 / solar) * m.c['solar'] + (1 / wind) * m.c['wind'] - m.p['power'] == 0
)
m.con7 = Constraint(expr=m.c['water'] <= 2.6)
m.con8 = Constraint(expr=m.c['solar'] <= 0.9)
m.con9 = Constraint(expr=m.c['wind'] <= 0.65)
m.con10 = Constraint(expr=m.d['oxygen'] <= 1.5)

m.obj = Objective(expr=-m.d['hydrogen'])

res_m = solver.solve(m)

In [252]:
print('hydrogen discharged:', m.d['hydrogen']())
print('oxygen discharged:', m.d['oxygen']())
print('water consumed:', m.c['water']())
print('solar consumed:', m.c['solar']())
print('wind consumed:', m.c['wind']())
print('power produced:', m.p['power']())

hydrogen discharged: 0.3191489361702128
oxygen discharged: 1.5
water consumed: 0.9414893617021276
solar consumed: 0.8936170212765958
wind consumed: 0.65
power produced: 1.2446808510638299


In [253]:
w = ConcreteModel()
# p1 - PV + PEM
# p2 - WF + PEM
w.x_ = Var(['p1', 'p2'], within=NonNegativeReals)

w.con1 = Constraint(expr=w.x_['p1'] <= 0.9 / (power * solar))
w.con2 = Constraint(expr=w.x_['p2'] <= 0.65 / (power * wind))
w.con3 = Constraint(expr=w.x_['p1'] + w.x_['p2'] <= min(2.6 / water, 1.5 / oxygen))

w.obj = Objective(expr=-(w.x_['p1'] + w.x_['p2']))
res_w = solver.solve(w)

In [257]:
d_hydrogen = w.x_['p1']() + w.x_['p2']()
d_oxygen = oxygen * (w.x_['p1']() + w.x_['p2']())
c_water = water * (w.x_['p1']() + w.x_['p2']())
c_solar = (power * solar) * w.x_['p1']()
c_wind = (power * wind) * w.x_['p2']()
p_power = power * (w.x_['p1']() + w.x_['p2']())

In [258]:
print('hydrogen discharged:', d_hydrogen)
print('oxygen discharged:', d_oxygen)
print('water consumed:', c_water)
print('solar consumed:', c_solar)
print('wind consumed:', c_wind)
print('power produced:', p_power)

hydrogen discharged: 0.3191489361702127
oxygen discharged: 1.4999999999999998
water consumed: 0.9414893617021276
solar consumed: 0.8936170212765956
wind consumed: 0.65
power produced: 1.2446808510638296
